# LAI/FAPAR Date Drill-Down
Pick a date, render a spatial map for that day. Adjust cell size and variable.

In [0]:
# Widgets
dbutils.widgets.text("SECRET_SCOPE", "climate-scope", "Secret Scope")
dbutils.widgets.text("AZURE_ACCOUNT_KEY_NAME", "AZURE_STORAGE_KEY", "Key Secret Name")
dbutils.widgets.text("AZURE_ACCOUNT_NAME_NAME", "AZURE_STORAGE_ACCOUNT", "Account Secret Name")
dbutils.widgets.text("CONTAINER", "climate-data-analysis", "ADLS Container")
dbutils.widgets.text("BASE_DIR", "climate_data/*/*.parquet", "Base Dir/Glob")
dbutils.widgets.dropdown("SOURCE", "adls", ["adls","local"], "Read From")
dbutils.widgets.text("PARQUET_LOCAL_GLOB", "ClimateRecords/parquet_output/*/*.parquet", "Local Parquet Glob")

dbutils.widgets.text("DATE", "2010-06-15", "Date (YYYY-MM-DD)")
dbutils.widgets.text("GRID_DEG", "0.10", "Spatial Cell Size (deg)")
dbutils.widgets.dropdown("VARIABLE", "LAI", ["LAI","FAPAR"], "Variable")

In [0]:
# Setup & read
import os
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import pandas as pd

SCOPE   = dbutils.widgets.get("SECRET_SCOPE")
ACC_SEC = dbutils.widgets.get("AZURE_ACCOUNT_NAME_NAME")
KEY_SEC = dbutils.widgets.get("AZURE_ACCOUNT_KEY_NAME")
account = dbutils.secrets.get(SCOPE, ACC_SEC)
key     = dbutils.secrets.get(SCOPE, KEY_SEC)
spark.conf.set(f"fs.azure.account.key.{account}.dfs.core.windows.net", key)

CONTAINER = dbutils.widgets.get("CONTAINER")
BASE_DIR  = dbutils.widgets.get("BASE_DIR").strip("/")
SOURCE    = dbutils.widgets.get("SOURCE")
LOCAL_GLOB = dbutils.widgets.get("PARQUET_LOCAL_GLOB")
DATE     = dbutils.widgets.get("DATE")
GRID_DEG = float(dbutils.widgets.get("GRID_DEG"))
VAR      = dbutils.widgets.get("VARIABLE")  # "LAI" or "FAPAR"

path = f"abfss://{CONTAINER}@{account}.dfs.core.windows.net/{BASE_DIR}" if SOURCE=="adls" else LOCAL_GLOB

# Read only the columns we need, per your schema
raw = spark.read.parquet(path).select("latitude", "longitude", "time", "LAI", "FAPAR")

# Convert epoch milliseconds → timestamp; also guard if any rows happen to be seconds
secs = F.when(F.col("time") > F.lit(100_000_000_000), F.col("time")/F.lit(1000.0)) \
        .otherwise(F.col("time").cast("double"))
sdf = (
    raw
    .withColumn("time", F.to_timestamp(F.from_unixtime(secs)))
    .filter(F.col("time").isNotNull())
    # OPTIONAL: drop early years if you want (uncomment next line)
    # .filter(F.year("time") >= 1982)
    .withColumnRenamed("LAI", "lai")
    .withColumnRenamed("FAPAR", "fapar")
)

# Ensure requested variable exists after normalization
var_col = VAR.lower()  # "lai" or "fapar"

# Filter to the requested date (YYYY-MM-DD)
day_sdf = sdf.where(F.to_date("time") == F.lit(DATE))

# Quick sanity checks
print(f"Records on {DATE}: {day_sdf.count()}")
day_sdf.agg(
    F.count(F.when(F.col(var_col).isNull(), 1)).alias(f"null_{var_col}")
).show()

display(day_sdf.select("time","latitude","longitude",var_col).limit(5))

Records on 1982-01-10: 1200
+----------+
|null_fapar|
+----------+
|      1150|
+----------+



time,latitude,longitude,fapar
1982-01-10T00:00:00Z,26.4749984741,-81.4749984741,null
1982-01-10T00:00:00Z,26.4749984741,-81.4749984741,null
1982-01-10T00:00:00Z,26.4749984741,-81.4249954224,null
1982-01-10T00:00:00Z,26.4749984741,-81.4249954224,null
1982-01-10T00:00:00Z,26.4749984741,-81.375,null


In [0]:
# Spatial hexbin for the selected day
cell = F.lit(GRID_DEG)
grid_sdf = (
    day_sdf
    .withColumn("lat_bin", F.floor(F.col("latitude")/cell)*cell)
    .withColumn("lon_bin", F.floor(F.col("longitude")/cell)*cell)
    .groupBy("lat_bin","lon_bin")
    .agg(F.mean(var_col).alias("value"))
    .select(F.col("lat_bin").alias("lat"), F.col("lon_bin").alias("lon"), "value")
)

grid = grid_sdf.toPandas()

if grid.empty:
    print(f"No data for {DATE}. Try another date or widen your selection.")
else:
    plt.figure(figsize=(6,6))
    plt.hexbin(grid["lon"], grid["lat"], C=grid["value"], gridsize=40)
    plt.title(f"{VAR} — {DATE} (cell {GRID_DEG}°)")
    plt.xlabel("Longitude"); plt.ylabel("Latitude")
    plt.colorbar(label=VAR)
    plt.tight_layout()
    display(plt.gcf()); plt.close()


---------------------------------------------------------------------------
SparkException                            Traceback (most recent call last)
File <command-5352098279229282>, line 12
      2 cell = F.lit(GRID_DEG)
      3 grid_sdf = (
      4     day_sdf
      5     .withColumn("lat_bin", F.floor(F.col("latitude")/cell)*cell)
   (...)
      9     .select(F.col("lat_bin").alias("lat"), F.col("lon_bin").alias("lon"), "value")
     10 )
---> 12 grid = grid_sdf.toPandas()
     14 if grid.empty:
     15     print(f"No data for {DATE}. Try another date or widen your selection.")

File /databricks/spark/python/pyspark/sql/connect/dataframe.py:2011, in DataFrame.toPandas(self)
   2009 def toPandas(self) -> "pandas.DataFrame":
   2010     query = self._plan.to_proto(self._session.client)
-> 2011     pdf, ei = self._session.client.to_pandas(query, self._plan.observations)
   2012     self._execution_info = ei
   2013     return pdf

File /databricks/spark/python/pyspark/sql/connect/cli